# IBM Data Science Capstone Project :: Week 4 & 5

**Analyzing Lahore City to find best town w.r.t nearby facilities**

Lahore is a city of Pakistan. There are few town that are old and few are built in recent past. People come here to seek jobs and to start their own business. For a person planning to settle in Lahore, he/ she will be having difficulty to choose the best town. More facilites surronded by the town may be a perfect choice for someone. This project aims to provide a overview of all the town w.r.t nearby venues like resturants, shopping malls etc.

**Data Used in Project**

*Data Prepration Phase* : Data of Towns in Lahore is available in Excel Format.

Geographical coordinates of the Town will be getting from online sources (OpenStreetMap or arcgis)

Obtain the venue data for the Towns from **Foursquare API**

Explore and cluster the Towns

Select the best Town

**Importing All Necessary Libraries**

In [14]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import geocoder
import requests

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print("Libraries imported.")

Libraries imported.


**Loading Data**

In [72]:
df_lhr = pd.read_excel("D:\\Documents\\Online Courses\\IBM Data Science\\Capstone Project\\Lahore_Towns.xlsx")

In [73]:
df_lhr.head()

,Town Names
0,Allama Iqbal Town
1,Badami Bagh
2,Bahria Town
3,Cavalry Ground
4,Daroghe Wala


**Getting Geographical Coordinates of Towns**

In [74]:
df_lhr_loc = pd.read_excel("D:\\Documents\\Online Courses\\IBM Data Science\\Capstone Project\\Lahore_Towns_Location_Info.xlsx")

In [75]:
df_lhr_loc.head()

,Town Names,Latitude,Longitude
0,Model Town,31.484382,74.326369
1,Faisal Town,31.476078,74.304168
2,Johar Town,31.467877,74.277992
3,Wapda Town,31.431386,74.264237
4,Township,31.447768,74.308218


**Combining Data Frames**

In [76]:
df_lhr = pd.merge(df_lhr,df_lhr_loc,how='left',on='Town Names')

In [77]:
df_lhr.head()

,Town Names,Latitude,Longitude
0,Allama Iqbal Town,31.511183,74.283185
1,Badami Bagh,31.601279,74.320618
2,Bahria Town,31.370790,74.183399
3,Cavalry Ground,31.499608,74.368200
4,Daroghe Wala,31.581263,74.398968


**Creating Map of Lahore**

In [79]:
address = 'Lahore, Punjab, Pakistan'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lahore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lahore City are 31.5656822, 74.3141829.


In [81]:
map_lhr = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood in zip(df_lhr['Latitude'], df_lhr['Longitude'], df_lhr['Town Names']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_lhr)  
    
map_lhr

**Foursquare API to explore the neighborhoods**

In [82]:
CLIENT_ID = 'K4DHSZU10COABVCQR1JGVSTQUONLHI25GBLTC1T3ZBIVHRRR'
CLIENT_SECRET = 'JTYEDWBA4FBNJAKSK3TQZ314RXJAGFWZ0QV2AWPNOJBGIKUM'
VERSION = '20180605'

In [83]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_lhr['Latitude'], df_lhr['Longitude'], df_lhr['Town Names']):
    
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [84]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(781, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Allama Iqbal Town,31.511183,74.283185,Kareem Block Market,31.504947,74.277878,Market
1,Allama Iqbal Town,31.511183,74.283185,Levi's® Factory Outlet,31.504356,74.280604,Clothing Store
2,Allama Iqbal Town,31.511183,74.283185,Rahim Store,31.521200,74.290413,Department Store
3,Allama Iqbal Town,31.511183,74.283185,Gulshan-e-Iqbal Park گلشن-اقبال پارک (گلشن-اقب...,31.512941,74.288990,Park
4,Allama Iqbal Town,31.511183,74.283185,OPTP,31.518230,74.292717,Fast Food Restaurant


In [85]:
venues_df.to_excel("D:\\Documents\\Online Courses\\IBM Data Science\\Capstone Project\\LHR_Venue_Info.xlsx",index=False)

In [86]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Allama Iqbal Town,21,21,21,21,21,21
Badami Bagh,11,11,11,11,11,11
Bahria Town,10,10,10,10,10,10
Cavalry Ground,51,51,51,51,51,51
DHA Phase 1,67,67,67,67,67,67
DHA Phase 2,33,33,33,33,33,33
DHA Phase 3,62,62,62,62,62,62
DHA Phase 4,50,50,50,50,50,50
DHA Phase 5,28,28,28,28,28,28


In [87]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 109 uniques categories.


**Analyzing Each Town**

In [88]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")


kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 


fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(781, 110)


,Neighborhoods,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bistro,Bookstore,Boutique,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Train Station,Women's Store,Zoo
0,Allama Iqbal Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Allama Iqbal Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Allama Iqbal Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Allama Iqbal Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Allama Iqbal Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(30, 110)


,Neighborhoods,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bistro,Bookstore,Boutique,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Train Station,Women's Store,Zoo
0,Allama Iqbal Town,0.000000,0.000000,0.000000,0.047619,0.00,0.095238,0.000000,0.047619,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Badami Bagh,0.000000,0.000000,0.000000,0.090909,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bahria Town,0.000000,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cavalry Ground,0.000000,0.039216,0.019608,0.019608,0.00,0.039216,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,DHA Phase 1,0.029851,0.000000,0.000000,0.014925,0.00,0.044776,0.000000,0.000000,0.000000,...,0.014925,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000
5,DHA Phase 2,0.030303,0.000000,0.000000,0.000000,0.00,0.030303,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,DHA Phase 3,0.016129,0.000000,0.016129,0.016129,0.00,0.032258,0.016129,0.016129,0.000000,...,0.016129,0.016129,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000
7,DHA Phase 4,0.020000,0.000000,0.020000,0.020000,0.00,0.000000,0.000000,0.020000,0.000000,...,0.020000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000
8,DHA Phase 5,0.000000,0.000000,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Daroghe Wala,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**Create a new DataFrame for Resturants data only**

In [90]:
lhr_rest = kl_grouped[["Neighborhoods","Pakistani Restaurant"]]

In [91]:
lhr_rest.head()

,Neighborhoods,Pakistani Restaurant
0,Allama Iqbal Town,0.000000
1,Badami Bagh,0.454545
2,Bahria Town,0.000000
3,Cavalry Ground,0.078431
4,DHA Phase 1,0.029851


**Clustering**

In [92]:
kclusters = 5

lhr_clustering = lhr_rest.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lhr_clustering)

kmeans.labels_[0:10]

array([1, 2, 1, 3, 1, 3, 1, 1, 1, 4])

In [93]:
lhr_merged = lhr_rest.copy()

lhr_merged["Cluster Labels"] = kmeans.labels_

In [97]:
lhr_merged.rename(columns={"Neighborhood": "Town Names"}, inplace=True)
lhr_merged.head()

,Town Names,Pakistani Restaurant,Cluster Labels
0,Allama Iqbal Town,0.000000,1
1,Badami Bagh,0.454545,2
2,Bahria Town,0.000000,1
3,Cavalry Ground,0.078431,3
4,DHA Phase 1,0.029851,1


In [98]:
lhr_merged = pd.merge(lhr_merged,df_lhr,on="Town Names",how='left')

In [99]:
lhr_merged.head()

,Town Names,Pakistani Restaurant,Cluster Labels,Latitude,Longitude
0,Allama Iqbal Town,0.000000,1,31.511183,74.283185
1,Badami Bagh,0.454545,2,31.601279,74.320618
2,Bahria Town,0.000000,1,31.370790,74.183399
3,Cavalry Ground,0.078431,3,31.499608,74.368200
4,DHA Phase 1,0.029851,1,31.482087,74.393934


In [101]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(lhr_merged['Latitude'], lhr_merged['Longitude'], lhr_merged['Town Names'], lhr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Cluster Examination**

In [102]:
lhr_merged.loc[lhr_merged['Cluster Labels'] == 0]

,Town Names,Pakistani Restaurant,Cluster Labels,Latitude,Longitude
11,Garden Town,0.157895,0,31.503582,74.325588
12,Gawalmandi,0.152174,0,31.572286,74.318651
15,Gulshan-e-Ravi,0.166667,0,31.550169,74.278721
21,Muslim Town,0.133333,0,31.518868,74.322817


In [103]:
lhr_merged.loc[lhr_merged['Cluster Labels'] == 1]

,Town Names,Pakistani Restaurant,Cluster Labels,Latitude,Longitude
0,Allama Iqbal Town,0.000000,1,31.511183,74.283185
2,Bahria Town,0.000000,1,31.370790,74.183399
4,DHA Phase 1,0.029851,1,31.482087,74.393934
6,DHA Phase 3,0.032258,1,31.472056,74.373290
7,DHA Phase 4,0.040000,1,31.464771,74.382612
8,DHA Phase 5,0.000000,1,31.463532,74.411504
13,Ghaziabad,0.000000,1,31.557726,74.395574
16,Harbanspura,0.000000,1,31.576498,74.427852
17,Johar Town,0.033333,1,31.467877,74.277992
18,Kot Khawaja Saeed,0.000000,1,31.591025,74.361789


In [104]:
lhr_merged.loc[lhr_merged['Cluster Labels'] == 2]

,Town Names,Pakistani Restaurant,Cluster Labels,Latitude,Longitude
1,Badami Bagh,0.454545,2,31.601279,74.320618


In [105]:
lhr_merged.loc[lhr_merged['Cluster Labels'] == 3]

,Town Names,Pakistani Restaurant,Cluster Labels,Latitude,Longitude
3,Cavalry Ground,0.078431,3,31.499608,74.368200
5,DHA Phase 2,0.060606,3,31.475152,74.401923
10,Faisal Town,0.073171,3,31.476078,74.304168
14,Gulberg,0.060000,3,31.510879,74.342400
19,Misri Shah,0.111111,3,31.588395,74.332970
20,Model Town,0.083333,3,31.484382,74.326369
22,Saddar Town,0.083333,3,31.538797,74.380816


In [106]:
lhr_merged.loc[lhr_merged['Cluster Labels'] == 4]

,Town Names,Pakistani Restaurant,Cluster Labels,Latitude,Longitude
9,Daroghe Wala,0.25,4,31.581263,74.398968
23,Sanda,0.20,4,31.560710,74.284453


**Final Findings**

As can be seen above, cluster 5 [Index 4] is having high percentage of Pakistani Resturants around them. A foodie person moving into Lahore would prefer this location.